## Data Encryption for Privacy Protection of PII (Personal Identifiable Information).  

# Cape-Privacy is python library supporting data transformations and collaborative privacy policies.

In [18]:
!pip install cape-privacy

     |████████████████████████████████| 48 kB 2.2 MB/s 
     |████████████████████████████████| 10.0 MB 7.3 MB/s 
     |████████████████████████████████| 20.1 MB 1.1 MB/s 
     |████████████████████████████████| 269 kB 55.2 MB/s 
     |████████████████████████████████| 13.7 MB 24.8 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=fba95e11f86ef5a9f4fe31035384551492277b0c4b404518f010b62e22604738
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built pyyaml
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Suc

In [ ]:
!git clone https://github.com/capeprivacy/cape-python.git

Cloning into 'cape-python'...
remote: Enumerating objects: 1308, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 1308 (delta 82), reused 72 (delta 42), pack-reused 1155
Receiving objects: 100% (1308/1308), 384.70 KiB | 607.00 KiB/s, done.
Resolving deltas: 100% (803/803), done.


In [ ]:
!cd cape-python

In [ ]:
!make bootstrap

make: *** No rule to make target 'bootstrap'.  Stop.


In [ ]:
import matplotlib.pylab as plt
import pandas as pd

import cape_privacy as cape
from cape_privacy.pandas import dtypes
from cape_privacy.pandas import transformations as tfms

plt.style.use('seaborn')


In [ ]:
df = pd.read_csv('/content/cape-python/examples/tutorials/credit/data/credit_with_pii.csv', parse_dates=["Application_date"])

In [ ]:
df.head()

,Name,City,Street_address,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,Robert Fitzgerald,Greenmouth,349 Alexander Coves Apt. 799,36964.79,2018-08-03,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,Daniel Kim,North Jessica,349 Jesse Park Suite 888,87884.39,2018-08-07,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,Frederick Jordan,Lake Erika,389 Graham Drive Suite 973,41157.73,2018-05-14,49,male,1,own,little,NaN,2096,12,education,good
3,Tara Rojas,East Jessica,01579 Ramirez Drives Apt. 587,36214.80,2018-04-30,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,Gail Donovan,Randyport,467 Christopher Well,41353.49,2018-05-27,53,male,2,free,little,little,4870,24,car,bad


In [ ]:
tokenize_name = tfms.Tokenizer()
tokenize_sex = tfms.Tokenizer(max_token_len=10)
perturb_age = tfms.NumericPerturbation(dtype=dtypes.Integer, min=-5, max=5)
perturb_application_date = tfms.DatePerturbation(frequency="DAY", min=-3, max=3)
redact_location = tfms.ColumnRedact(columns=["Street_address", "City"])
round_salary = tfms.NumericRounding(dtype=dtypes.Float, precision=-3)


In [ ]:
def mask(df):
    df = df.copy()
    df['Name'] = tokenize_name(df['Name'])
    df['Sex'] = tokenize_sex(df['Sex'])
    df['Age'] = perturb_age(df['Age'])
    df['Application_date'] = perturb_application_date(df['Application_date'])
    df['Salary'] = round_salary(df['Salary'])
    df = redact_location(df)
    return df

caped_df = mask(df)
caped_df.head()

,Name,Salary,Application_date,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,220647044fe3ec5be919a49b86fb24fd41287c44ba3bdc...,37000.0,2018-08-05,66,ccde7a9016,2,own,NaN,little,1169,6,radio/TV,good
1,345fff4dfd20866b73d05b29d6ed15548e9d0b35ff5991...,88000.0,2018-08-05,23,5c5d1c1eff,2,own,little,moderate,5951,48,radio/TV,bad
2,e639532a22bfc33704dd728849d14ff4cfd377562acd93...,41000.0,2018-05-15,52,ccde7a9016,1,own,little,NaN,2096,12,education,good
3,448679a8cbde30dfbcec610d74c6f12e03b9ffd96db32e...,36000.0,2018-04-30,47,ccde7a9016,2,free,little,little,7882,42,furniture/equipment,good
4,822aeacc36e095f85e58ed0f87b76e8589b566122dd67c...,41000.0,2018-05-28,54,ccde7a9016,2,free,little,little,4870,24,car,bad


#Using Hashlib Builtin Python Library

In [3]:
import hashlib

# Encode our string using UTF-8 default 
stringToHash = 'user@email.com'.encode()

In [4]:
# Hash using SHA-256 and print
print('Email (SHA-256): ', hashlib.sha256(stringToHash).hexdigest())

Email (SHA-256):  0925f997eb0d742678f66d2da134d15d842d57722af5f7605c4785cb5358831b


In [6]:
import pandas as pd

df = pd.read_csv('/content/BankChurners.csv', usecols=['CLIENTNUM', 'Customer_Age', 'Gender', 'Attrition_Flag', 'Total_Trans_Amt'])
df.head()

,CLIENTNUM,Attrition_Flag,Customer_Age,Gender,Total_Trans_Amt
0,768805383,Existing Customer,45,M,1144
1,818770008,Existing Customer,49,F,1291
2,713982108,Existing Customer,51,M,1887
3,769911858,Existing Customer,40,F,1171
4,709106358,Existing Customer,40,M,816


In [7]:
# Convert column to string
df['CLIENTNUM'] = df['CLIENTNUM'].astype(str)

# Apply hashing function to the column
df['CLIENTNUM_HASH'] = df['CLIENTNUM'].apply(
    lambda x: 
        hashlib.sha256(x.encode()).hexdigest()
)

In [8]:
df[['CLIENTNUM', 'CLIENTNUM_HASH']].head()


,CLIENTNUM,CLIENTNUM_HASH
0,768805383,c9bbef56f9d8292cb3cfa8ae91f9b9167390e6e4b514d5...
1,818770008,7996e2340d70489252370a5df035ec99381c8344cc3511...
2,713982108,6fb53dbc743724e086243b5bc288df62b4a6dc1b8bde92...
3,769911858,f86b86a1047317685f29c399059b199858685faf5ec6a8...
4,709106358,0d239470b0cb57e110cf60bc3865344ee2cdced6e3acdc...


#Using Cryptography Library with Encryption Keys

In [9]:
!pip install cryptography

     |████████████████████████████████| 3.6 MB 5.1 MB/s 


In [10]:
import cryptography
from cryptography.fernet import Fernet

# Generate base64 encoder String key 
key = Fernet.generate_key()
print(key)

b'noucBDzd1uzH4I5PFcVK2aFTJYxEXgl-vpN7A4T-E28='


In [11]:
import cryptography
from cryptography.fernet import Fernet

key = Fernet.generate_key()
print(key)
 
# Save as a key to encrypt
file = open('key.key', 'wb') #wb = write bytes
file.write(key)
file.close()

b'HRwPxs79hGBBKLtpRkIcOaLlyTF1hNVSYRbKdLSYUOQ='


In [12]:
# Retrive key
file = open('key.key', 'rb') # rb = read bytes
key  = file.read()
file.close()
print (key)

b'HRwPxs79hGBBKLtpRkIcOaLlyTF1hNVSYRbKdLSYUOQ='


Creating Encryption key with password

In [13]:
import os
os.urandom(16)

b'W\x97\xf6\xfd\xdb\xfd\x7f\xac\xf2\xb7s\x0f\xbd\x8b(\xde'

In [14]:
import base64
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

password_provided = 'password'
password = password_provided.encode()

salt = b"W\x97\xf6\xfd\xdb\xfd\x7f\xac\xf2\xb7s\x0f\xbd\x8b(\xde"

kdf = PBKDF2HMAC(algorithm=hashes.SHA256(),
                length=32,
                salt=salt,
                iterations=100000,
                backend=default_backend())

key = base64.urlsafe_b64encode(kdf.derive(password))
print(key)

b'sUKAw8m89-0BFUXFo1Vk1a8BY9AemPP03hPGtv9nFbA='


In [15]:
# Get the key from the file
file = open('key.key', 'rb')
key = file.read()
file.close()

#  Open the file to encrypt
with open('/content/BankChurners.csv', 'rb') as f:
    data = f.read()

fernet = Fernet(key)
encrypted = fernet.encrypt(data)

# Write the encrypted file
with open('secretdf.csv.encrypted', 'wb') as f:
    f.write(encrypted)

# Decryption process

In [17]:
# Get the key from the file
file = open('key.key', 'rb')
key = file.read()
file.close()

#  Open the file to decrypt
with open('/content/secretdf.csv.encrypted', 'rb') as f:
    data = f.read()

fernet = Fernet(key)
encrypted = fernet.decrypt(data)

# Open the decrypted file
with open('secretdf.csv.decrypted', 'wb') as f:
    f.write(encrypted)

# Reference Links
https://github.com/capeprivacy/cape-python
https://towardsdatascience.com/encrypting-your-data-9eac85364cb
https://towardsdatascience.com/anonymise-sensitive-data-in-a-pandas-dataframe-column-with-hashlib-8e7ef397d91f
https://medium.com/dropoutlabs/cape-python-apply-privacy-enhancing-techniques-to-protect-sensitive-data-in-pandas-and-spark-e0bf8c0d55db